## Type providers

Continuing with reading structured data files, sometimes it's not possible (or a lot of work) to _parse_ the data. Let's see an example where this happens:

In [1]:
let readFile(fileName: string) =  
    let lines = File.ReadAllLines(fileName)
    lines     

In this case, we take data from a file with Beatles songs

In [2]:
let beatlesFile = "../data/The Beatles songs dataset.csv"

let songs = readFile(beatlesFile)

In [3]:
songs.GetType()

System.String[]

In [5]:
printfn "%A" songs[0]
printfn "%A" songs[1]

"Title,Year,Album.debut,Duration,Other.releases,Genre,Songwriter,Lead.vocal,Top.50.Billboard"
"12-Bar Original,1965,Anthology 2,174,0,Blues,"Lennon, McCartney, Harrison and Starkey",,-1"


In [6]:
songs[0..20]
|> Seq.iteri  (fun i s->  printfn $"{i}: {s}")

0: Title,Year,Album.debut,Duration,Other.releases,Genre,Songwriter,Lead.vocal,Top.50.Billboard
1: 12-Bar Original,1965,Anthology 2,174,0,Blues,"Lennon, McCartney, Harrison and Starkey",,-1
2: A Day in the Life,1967,Sgt. Pepper's Lonely Hearts Club Band,335,12,"Psychedelic Rock, Art Rock, Pop/Rock",Lennon and McCartney,Lennon and McCartney,-1
3: A Hard Day's Night,1964,UK: A Hard Day's Night US: 1962-1966,152,35,"Rock, Electronic, Pop/Rock",Lennon,"Lennon, with McCartney",8
4: A Shot of Rhythm and Blues,1963,Live at the BBC,104,0,"R&B, Pop/Rock",Thompson,Lennon,-1
5: A Taste of Honey,1963,UK: Please Please Me US: The Early Beatles,163,29,"Pop/Rock, Jazz, Stage&Screen","Scott, Marlow",McCartney,-1
6: Across the Universe,1968,Let It Be,230,19,"Psychedelic folk, Pop/Rock",Lennon,Lennon,-1
7: Act Naturally,1965,UK: Help! US: Yesterday and Today,139,14,"Country, Pop/Rock","Russell, Morrison",Starkey,50
8: Ain't She Sweet,1961,Anthology 1,150,9,Pop/Rock,"Yellen, Ager",Lennon,41
9: All I've Go

In [9]:
let song =   songs[2].Split(',')
printfn "%A" songs[2]
printfn "%A" song

"A Day in the Life,1967,Sgt. Pepper's Lonely Hearts Club Band,335,12,"Psychedelic Rock, Art Rock, Pop/Rock",Lennon and McCartney,Lennon and McCartney,-1"
[|"A Day in the Life"; "1967"; "Sgt. Pepper's Lonely Hearts Club Band"; "335";
  "12"; ""Psychedelic Rock"; " Art Rock"; " Pop/Rock""; "Lennon and McCartney";
  "Lennon and McCartney"; "-1"|]


Vemos la dificultad en el correcto _parseo_ de los datos: existen campos con `,` dentro de `""`, con lo cual un simple `Split` no funciona adecuadamente, y es necesario una programación más delicada para solucionar estos problemas. 

In [7]:
song.Length

3

One can consult other _parsers_, which are everywhere, for example [on this page](https://www.joelverhagen.com/blog/2020/12/fastest-net-csv-parsers), but, thanks to the static data type, there is a module that can solve the problem for us, through _type providers_.

### Type Providers

A _type provider_ is a library that allows us to deal with particular types of data:

- [CSV type provider](http://fsprojects.github.io/FSharp.Data/library/CsvProvider.html).
- [Html type provider](https://fsprojects.github.io/FSharp.Data/library/HtmlProvider.html).
- [Json type provider](https://fsprojects.github.io/FSharp.Data/library/JsonProvider.html).

are some examples.

The `FSharp.Data` library is the one we will use to learn how to read these data types. In a notebook it is imported as follows:

In [10]:
#r "nuget: FSharp.Data"

open FSharp.Data

Installed Packages FSharp.Data, 6.2.0

A _type provider_ generates a data type from the information it reads from a file. This occurs at compile time. At the time of executing the code, the type that was created can be used to process the data

In [13]:
type SongsTypeProvider = FSharp.Data.CsvProvider<"../data/The Beatles songs dataset.csv", HasHeaders=true>

The compiler (and the `FSharp.Data` library) build the _type provider_ using the file "../data/The Beatles songs dataset.csv" as a template, discovering the structure of the data.

You can get the data itself with:

In [14]:
let songs = SongsTypeProvider.GetSample()

Here we use the same file to create the type and to get the data. However, you could use two different files, one as a template and one with the data. In that case, we call the `.Load` method:

```fsharp
type SongsTypeProvider = FSharp.Data.CsvProvider<"myTemplateDataFile.csv", HasHeaders=true>
let songs = SongsTypeProvider.Load("myRealDataFile.csv")
```


When creating the data type, the _type provider_ creates the fields to be able to access the information, for example

In [15]:
songs.Headers 

Value,"[ Title, Year, Album.debut, Duration, Other.releases, Genre, Songwriter, Lead.vocal, Top.50.Billboard ]"


gives us the headers of each column of the data. The data itself is found in the `.Rows` field:

In [16]:
songs.Rows
|> Seq.take 20 
|> Seq.iteri  (fun i s ->  printfn $"{i}: {s}")

0: (12-Bar Original, 1965, Anthology 2, 174, 0, Blues, Lennon, McCartney, Harrison and Starkey, , -1)
1: (A Day in the Life, 1967, Sgt. Pepper's Lonely Hearts Club Band, 335, 12, Psychedelic Rock, Art Rock, Pop/Rock, Lennon and McCartney, Lennon and McCartney, -1)
2: (A Hard Day's Night, 1964, UK: A Hard Day's Night US: 1962-1966, 152, 35, Rock, Electronic, Pop/Rock, Lennon, Lennon, with McCartney, 8)
3: (A Shot of Rhythm and Blues, 1963, Live at the BBC, 104, 0, R&B, Pop/Rock, Thompson, Lennon, -1)
4: (A Taste of Honey, 1963, UK: Please Please Me US: The Early Beatles, 163, 29, Pop/Rock, Jazz, Stage&Screen, Scott, Marlow, McCartney, -1)
5: (Across the Universe, 1968, Let It Be, 230, 19, Psychedelic folk, Pop/Rock, Lennon, Lennon, -1)
6: (Act Naturally, 1965, UK: Help! US: Yesterday and Today, 139, 14, Country, Pop/Rock, Russell, Morrison, Starkey, 50)
7: (Ain't She Sweet, 1961, Anthology 1, 150, 9, Pop/Rock, Yellen, Ager, Lennon, 41)
8: (All I've Got to Do, 1963, UK: With the Beatles 

From the column headers, the _type provider_ builds the fields corresponding to each piece of data:

In [17]:
songs.Rows
|> Seq.take 10 
|> Seq.iteri  (fun i s ->  printfn $"{i}: {s.Title} by {s.Songwriter} ({s.Year})")

0: 12-Bar Original by Lennon, McCartney, Harrison and Starkey (1965)
1: A Day in the Life by Lennon and McCartney (1967)
2: A Hard Day's Night by Lennon (1964)
3: A Shot of Rhythm and Blues by Thompson (1963)
4: A Taste of Honey by Scott, Marlow (1963)
5: Across the Universe by Lennon (1968)
6: Act Naturally by Russell, Morrison (1965)
7: Ain't She Sweet by Yellen, Ager (1961)
8: All I've Got to Do by Lennon (1963)
9: All My Loving by McCartney (1963)


In [18]:
songs.Rows
|> Seq.take 20 
|> Seq.iteri  (fun i s ->  printfn $"{i}: {s.Title} by >{s.``Lead.vocal``}<")

0: 12-Bar Original by ><
1: A Day in the Life by >Lennon and McCartney<
2: A Hard Day's Night by >Lennon, with McCartney<
3: A Shot of Rhythm and Blues by >Lennon<
4: A Taste of Honey by >McCartney<
5: Across the Universe by >Lennon<
6: Act Naturally by >Starkey<
7: Ain't She Sweet by >Lennon<
8: All I've Got to Do by >Lennon<
9: All My Loving by >McCartney<
10: All Things Must Pass by >Harrison<
11: All Together Now by >McCartney, with Lennon<
12: All You Need Is Love by >Lennon<
13: And I Love Her by >McCartney<
14: And Your Bird Can Sing by >Lennon<
15: Anna (Go to Him) by >Lennon<
16: Another Girl by >McCartney<
17: Any Time at All by >Lennon, with McCartney<
18: Ask Me Why by >Lennon<
19: Baby It's You by >Lennon<


In [19]:
songs.Rows 
|> Seq.filter (fun r -> r.``Top.50.Billboard``=1)
|> Seq.iter (fun r -> printfn $"Name:{r.Title} position {r.``Top.50.Billboard``}")


Name:Hey Jude position 1


#### Choosing separators

Separators can be specified when creating the type:
```fsharp
CsvProvider<"../data/AirQuality.csv", Separators=";,">
```

### Missing data

The _type provider_ has [certain rules for dealing with missing data](https://fsprojects.github.io/FSharp.Data/library/CsvProvider.html#Controlling-the-column-types). For example, if the data expected in some column is a number, but the file contains `NaN`, when creating the data the _type provider will report it as `Double.NaN`.

On the other hand, [we can specify which `strings` we want to be converted to `Nan`](https://fsprojects.github.io/FSharp.Data/library/CsvProvider.html#Missing-values):

```fsharp 
CsvProvider<"X,Y,Z\nthis,that,1.0", MissingValues="this,that">
    .GetSample()
    .Rows
```

Also, if we prefer not to use the _type provider_ rules, we use `PreferOptionals=true` to make it generate `option` types in the case
of missing data:


In [20]:
type SongsTypeProviderOpt = FSharp.Data.CsvProvider<"../data/The Beatles songs dataset.csv", HasHeaders=true, PreferOptionals=true>

let songsWithOpt = SongsTypeProviderOpt.GetSample()

songsWithOpt.Rows
|> Seq.take 5
|> Seq.iteri  (fun i s ->  printfn $"{i}: {s}")

0: (12-Bar Original, 1965, Some(Anthology 2), 174, 0, Some(Blues), Lennon, McCartney, Harrison and Starkey, , -1)
1: (A Day in the Life, 1967, Some(Sgt. Pepper's Lonely Hearts Club Band), 335, 12, Some(Psychedelic Rock, Art Rock, Pop/Rock), Lennon and McCartney, Some(Lennon and McCartney), -1)
2: (A Hard Day's Night, 1964, Some(UK: A Hard Day's Night US: 1962-1966), 152, 35, Some(Rock, Electronic, Pop/Rock), Lennon, Some(Lennon, with McCartney), 8)
3: (A Shot of Rhythm and Blues, 1963, Some(Live at the BBC), 104, 0, Some(R&B, Pop/Rock), Thompson, Some(Lennon), -1)
4: (A Taste of Honey, 1963, Some(UK: Please Please Me US: The Early Beatles), 163, 29, Some(Pop/Rock, Jazz, Stage&Screen), Scott, Marlow, Some(McCartney), -1)


### Html Provider

We can also get the data from a web page using an Html provider:

In [21]:
[<Literal>]
let url = """https://en.wikipedia.org/wiki/List_of_songs_recorded_by_the_Beatles"""

type WebSongsTypeProvider = FSharp.Data.HtmlProvider<url>

let songs = WebSongsTypeProvider.Load("https://en.wikipedia.org/wiki/List_of_songs_recorded_by_the_Beatles")

In [22]:
songs.Tables

FSharp.Data.Runtime.BaseTypes.HtmlList`1[System.String] Name Languages Values [ Deutsch, Español, فارسی, Français, ქართული, Magyar, 日本語, Polski, Português, Русский ] Html \n \n \n Deutsch \n... _Print <ul class="minerva-languages">
 <li class="interlanguage-link interwiki-de mw-list-item">
 <a href="https://de.wikipedia.org/wiki/Liste_der_Lieder_der_Beatles" title="Liste der Lieder der Beatles – German" lang="de" hreflang="de" class="interlanguage-link-target">
 <span>Deutsch</span>
 </a>
 </li>
 <li class="interlanguage-link interwiki-es mw-list-item">
 <a href="https://es.wikipedia.org/wiki/Anexo:Canciones_de_The_Beatles" title="Anexo:Canciones de The Beatles – Spanish" lang="es" hr...

In [23]:
songs.Tables.``Main songsEdit 3``.Rows
|> Seq.map (fun r -> r.Song, r.Year)
|> Seq.filter (fun (s,y) -> y=1968)
|> Seq.iter (fun (s,y) -> printfn $"Name:%s{s} Year:{y}")


Name:"Back in the U.S.S.R." Year:1968
Name:"Birthday" Year:1968
Name:"Blackbird" Year:1968
Name:"The Continuing Story of Bungalow Bill" Year:1968
Name:"Cry Baby Cry" Year:1968
Name:"Dear Prudence" Year:1968
Name:"Don't Pass Me By" Year:1968
Name:"Everybody's Got Something to Hide Except Me and My Monkey" Year:1968
Name:"Glass Onion" Year:1968
Name:"Good Night" Year:1968
Name:"Happiness Is a Warm Gun" Year:1968
Name:"Helter Skelter" Year:1968
Name:"Hey Jude" # Year:1968
Name:"Honey Pie" Year:1968
Name:"I Will" Year:1968
Name:"I'm So Tired" Year:1968
Name:"The Inner Light" # Year:1968
Name:"Julia" Year:1968
Name:"Lady Madonna" # Year:1968
Name:"Long, Long, Long" Year:1968
Name:"Martha My Dear" Year:1968
Name:"Mother Nature's Son" Year:1968
Name:"Ob-La-Di, Ob-La-Da" Year:1968
Name:"Piggies" Year:1968
Name:"Revolution"[m]# Year:1968
Name:"Revolution 1"[n] Year:1968
Name:"Revolution 9"[o] Year:1968
Name:"Rocky Raccoon" Year:1968
Name:"Savoy Truffle" Year:1968
Name:"Sexy Sadie" Year:1968
Nam

### Json Provider

Finally, there is a _type provider_ to read data in JSON (JavaScript Object Notation) format, which is standard in the transmission of information on the Internet.

In [24]:
[<Literal>]
let tvUrl = "https://raw.githubusercontent.com/mganitombalak/training/master/DATA/tv-shows.json"

In [25]:
type TvListing = JsonProvider<tvUrl>
let tvListing = TvListing.GetSamples()                                   


In [27]:
tvListing.Length

240

In [26]:
tvListing
|> Seq.map (fun t -> (t.Name,t.Rating.Average))
|> Seq.sortByDescending (fun (n,a) -> a)
|> Seq.take 20
|> Seq.iter (fun (n,a) -> printfn $"{n}: {a}")

"Game of Thrones": Some(9.4)
"Rick and Morty": Some(9.4)
"Breaking Bad": Some(9.3)
"The Wire": Some(9.3)
"Firefly": Some(9.3)
"Stargate SG-1": Some(9.3)
"Berserk": Some(9.2)
"Person of Interest": Some(9)
"Fargo": Some(9)
"House": Some(9)
"Banshee": Some(9)
"The Newsroom": Some(8.9)
"Fringe": Some(8.9)
"Battlestar Galactica": Some(8.9)
"Stargate Atlantis": Some(8.9)
"Vikings": Some(8.8)
"Boardwalk Empire": Some(8.8)
"Justified": Some(8.8)
"Bob's Burgers": Some(8.8)
"CSI: Crime Scene Investigation": Some(8.8)


In [28]:
tvListing
|> Seq.map (fun t -> (t.Name,t.Rating.Average))
|> Seq.choose (fun (n,a) -> a)
|> Seq.length

232